# Pipeline

In [1]:
import findspark, pyspark
from pyspark.sql import SparkSession
findspark.init()
spark = SparkSession.builder.appName("fpgrowth").getOrCreate()

24/04/03 17:26:18 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.0.108 instead (on interface wlo1)
24/04/03 17:26:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 17:26:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/03 17:26:19 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
iris = spark.read.csv("../0_data/iris.csv", header=True, inferSchema=True, sep=",")
print(iris.count())
iris.show(5)

In [ ]:
vector_assembler = VectorAssembler(inputCols=["sepallength", "sepalwidth", "petallength", "petalwidth"], outputCol="independant")
indexer = StringIndexer(inputCol="class", outputCol="dependant")

In [ ]:
mlp = MultilayerPerceptronClassifier(featuresCol="independant", labelCol="dependant", maxIter=100, layers=[4, 5, 4, 3])

In [ ]:
pipeline = Pipeline(stages=[vector_assembler, indexer, mlp])
model = pipeline.fit(iris)

In [ ]:
prediction = model.transform(iris)
prediction.show(5)

+-----------+----------+-----------+----------+-----------+-----------------+---------+--------------------+--------------------+----------+
|sepallength|sepalwidth|petallength|petalwidth|      class|      independant|dependant|       rawPrediction|         probability|prediction|
+-----------+----------+-----------+----------+-----------+-----------------+---------+--------------------+--------------------+----------+
|        5.1|       3.5|        1.4|       0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|      0.0|[70.6663493867936...|[1.0,5.7679087317...|       0.0|
|        4.9|       3.0|        1.4|       0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|      0.0|[70.6663493867936...|[1.0,5.7679087317...|       0.0|
|        4.7|       3.2|        1.3|       0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|      0.0|[70.6663493867936...|[1.0,5.7679087317...|       0.0|
|        4.6|       3.1|        1.5|       0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|      0.0|[70.6663493867936...|[1.0,5.7679087317...|       0.0|
|        5.0|

In [ ]:
performance = MulticlassClassificationEvaluator(labelCol="dependant", predictionCol="prediction", metricName="accuracy")
accuracy = performance.evaluate(prediction)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9933333333333333
